## 참조
논문 리뷰 : https://www.youtube.com/watch?v=AA621UofTUA 

# Seq2Seq
<img src = "./seq2seq.png">

- encoder - decoder 구조로 구성되어 있다.


여기서 encoder는 입력 시퀀스를 하나의 백터 표현으로 압축하고, decoder는 이 벡터 표현을 통해서 출력 시퀀스를 만들어낸다.


인코더에서는 매번 단어가 입력될 때마다 hidden stay 값을 받아 다음 히든스테이로 전달하며 최종 히든스테이에 저장된 문장의 정보를 context vector에서 고정된 크기로 압축하여 디코더로 전달한다.


하지만 고정된 크기의 context vector로 압축하려고 하면 문장의 길고 짧은 다양한 변수에 전체 성능이 떨어지는 병목현상이 발생한다.


이러한 문제룰 해결하고자 매번 소스 문장에서의 출력전부를 입력으로 받아 사용하는 Attention이 나왔다.

# Attention 
<img src = "./attention1.png">

인코딩을 출력하여 히든스테이가 나올 때마다 별도의 배열에 다 기록해서 이러한 값들을 참고할 수 있도록 반영.


즉, context vector 뿐만 아니라 매번 출력할때마다 소스문장에서 나온 출력값들을 모두 참고하겠다는 뜻

### Attention 작동 원리

<img src = "./attention2.png">

- i = 현재의 디코더가 처리 중인 인덱스
- j = 각각의 인코더 출력 인덱스

- 에너지
    - 매번 디코더가 출력값을 만들때 디코더의 이전 출력 정보와 인코더에서의 모든 출력을 비교해서 에너지값을 구한다.
    - 즉, 어떤 h값과 많은 연관성을 가지는지 그 에너지 값을 구하는 것
    
    
- 가중치
    - 이 에너지 값에 소프트맥스를 취해 확률값으로 구한다. 


다시 말해 현재 디코더의 히든스테이트를 만들기 위해 이전 히드스테이트와 인코더의 모든 히든스테이 값을 묶어 에너지 값을 구한 뒤 거기에 소프트맥스를 취해 비율 값을 구한다.이러한 가중치 값을 각각의 h값에 곱해서 전부 더해준 결과를 실제 디코더의 입력으로 활용한다.
    



# Transformer

- RNN, CNN의 약점으로 많이 언급되었던 병렬처리가 어려움을 해결
- 순차적으로 입력받는 것이 아닌 Sequence를 한번에 넣음으로써 병렬처리가 가능하면서 Attentionn 등의 구조를 통해 어떤 부분이 중요한지 전달
- RNN이나 CNN을 전혀 사용하지 않고 문장 내에 단어들의 순서의 정보를 주기 위해 대신 Positional Encoding을 사용한다.
- 인코더와 디코더로 구성되어있다.
    - Attention 과정을 여러 레이어에서 반복한다.
    
<img src = "./transformer.png">

### 1. 인코더

RNN기반의 아키텍처는 RNN 사용으로 각각의 단어가 RNN으로 들어갈 때 순서에 맞게 들어가 순서에 대한 정보를 가진다.

Transformer는 위치에 대한 정보(어떤 단어가 앞에오고, 뒤에 오는지)를 주기 위해서 위치 정보를 포함하고 있는 임베딩을 사용해야 한다.


#### 1) Positional Encoding
<img src = "./positional.png">

- Input Embedding과 같은 크기를 가지는 별도의 위치 정보를 갖고 있는 Positional Encoding을 넣어서 각각 더함으로써 각각의 단어가 어떤 순서를 가지는지 네트워크가 알 수 있도록 만들어준다.

- 따라서, Multi-head Attention이 받는 값은 (입력문장에 대한 정보 + 위치에 대한 정보)이다.

- 인코더 파트에서 수행하는 어탠션은 문장을 구성하는 각각의 단어가 서로에게 어떠한 연관성이 있는지 어텐션 스코어를 구한다.

- 그래서 그러한 입력을 받아서 각각의 단어들을 이용해서 Attention을 수행

#### 2) Attention
<img src = "./attention.png">

인코더와 디코더는 Multi-Head Attention 레이어를 사용한다.

어텐션을 위한 세 가지 입력 요소

- 쿼리 : 현재 시점의 token
- 키 : attention을 구하고자 하는 대상 token
- 값

예) I am a teacher라면 쿼리는 : I , 키 : I am a teacher 각각의 단어

즉, 어떠한 단어가 다른 단어들과 어떠한 가중치를 가지는지 구하는 것.

행렬곱, 스케일, 마스크, 소프트맥스를 통해 비율을 구하고 실제 Value값과 곱하여 가중치가 적용된 어탠션 벨루값이 구해진다.

이를 h개의 서로 다른 쿼리, 키, 값을 가진다.

→ 서로 다른 어탠션 컨셉을 학습하도록 만들어서 다양한 특징을 학습하도록 유도한다.

그 후 Concat한다. 

입력값과 출력값의 dimension은 같아야 하므로 concat으로 붙이고 Linear 레이어를 거쳐서 아웃풋을 낸다.

#### 3) Residual Learning(잔여학습)
이후 **잔여학습(Residual Learning)**을 사용한다

어떠한 값을 레이어를 거쳐 반복으로 갱신하는 것이 아닌 특정 레이어를 건너뛰어서 복사가 된 값을 그대로 입력할 수 있도록 만드는 것

이렇게 함으로써 전체 네트워크는 기존 정보를 입력받으면서 추가적으로 잔여된 부분만 학습함으로 학습 난이도가 낮고 초기 모델 수렴속도가 높게 되어 전반적으로 다양한 네트워크에 대해서 성능이 좋아질 수 있다.

#### 4) Add + Norm
Attention으로 받은 값과 Residual Learning으로 받은 값을 같이 더해서 노멀레이션을 수행하여 내보낸다. 

<img src = "./add.png">

이렇게 Attention과 Normalization 과정을 반복

- 각 레이어는 서로 다른 파라미터를 가진다. Layer1과 Layer2에 사용되는 Attention과 Feedforward의 파라미터는 다르다.

#### 5) FeedForward Layer

입력 벡터를 선형 변환하고, 그 다음 비선형 함수인 ReLU를 적용한 후 다시 선형 변환을 수행한다. 이를 통해 모델이 보다 복잡한 함수를 학습할 수 있도록 한다. 즉, 입력 정보를 더 추상화된 표현으로 변환하고, 모델의 성능 향상을 도와주는 역할을 한다.

### 2. 디코더

<img src = "./decoder.png">
가장 마지막에 인코더 레이어에서 나온 출력값이 디코더에 들어간다.

디코더에서는Attentinon을 두개 사용

첫번째는 Self Attention으로 인코더 파트와 마찬가지로 각각의 단어들이 서로가 서로에게 어떠한 가중치를 두는지 구하도록 만든다.

두번째 Attention은 인코더에 대한 정보를 어텐션할 수 있도록 만든다. 각각의 출력되고 있는 단어가 소스 문장에서의 어떤 단어와 연관성이 있는지 구해준다